In [ ]:
from mtcnn import MTCNN
import cv2
import numpy as np
import tensorflow as tf
from pygame import mixer
from collections import deque
import time

In [ ]:
frame_counter = 0
prev_time = time.time()

Load MTCNN detector

In [ ]:
detector = MTCNN()

Load pre-trained eye closure classification model

In [ ]:
model = tf.keras.models.load_model('finalModel_Augmented.keras')

Alarm setup

In [ ]:
mixer.init()
try:
    alarm = mixer.Sound('alarm.wav')
except:
    print("Warning: alarm.wav not found.")
    alarm = None

Parameters

In [ ]:
score = 0
SCORE_THRESHOLD = 5
eye_status_history = deque(maxlen=3)
font = cv2.FONT_HERSHEY_SIMPLEX

In [ ]:
def preprocess_eye(img):
    img = cv2.resize(img, (64, 64))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    #FPS LOGIC
    frame_counter += 1
    if frame_counter == 10:
        curr_time = time.time()
        elapsed_time = curr_time - prev_time
        fps = frame_counter / elapsed_time
        print(f"FPS: {fps:.2f}")
        frame_counter = 0
        prev_time = curr_time
    faces = detector.detect_faces(frame)
    for face in faces:
        bbox = face['box']
        keypoints = face['keypoints']
        left_eye_x, left_eye_y = keypoints['left_eye']
        right_eye_x, right_eye_y = keypoints['right_eye']
        eye_width = abs(left_eye_x - right_eye_x)

        # Get ROIs for left and right eye
        lx1 = max(0, int(left_eye_x - eye_width / 2))
        ly1 = max(0, int(left_eye_y - eye_width / 2))
        lx2 = min(frame.shape[1], int(left_eye_x + eye_width / 2))
        ly2 = min(frame.shape[0], int(left_eye_y + eye_width / 2))
        rx1 = max(0, int(right_eye_x - eye_width / 2))
        ry1 = max(0, int(right_eye_y - eye_width / 2))
        rx2 = min(frame.shape[1], int(right_eye_x + eye_width / 2))
        ry2 = min(frame.shape[0], int(right_eye_y + eye_width / 2))
        try:
            left_eye_region = frame[ly1:ly2, lx1:lx2]
            right_eye_region = frame[ry1:ry2, rx1:rx2]

            # left_eye_gray = cv2.cvtColor(left_eye_region, cv2.COLOR_BGR2GRAY)
            # right_eye_gray = cv2.cvtColor(right_eye_region, cv2.COLOR_BGR2GRAY)

            # Preprocess
            left_input = preprocess_eye(left_eye_region)
            right_input = preprocess_eye(right_eye_region)
            lpred = model.predict(left_input, verbose=0)[0][0]
            rpred = model.predict(right_input, verbose=0)[0][0]
            l_state = 1 if lpred > 0.5 else 0
            r_state = 1 if rpred > 0.5 else 0
            eyes_closed = (l_state == 0 and r_state == 0)
            eye_status_history.append(1 if eyes_closed else 0)
            if sum(eye_status_history) >= 2:
                score += 1
            else:
                score -= 1
            score = max(0, score)

            # Alarm logic
            if score > SCORE_THRESHOLD:
                if alarm and not mixer.get_busy():
                    alarm.play()
                cv2.putText(frame, "DROWSY!", (10, 60), font, 1, (0, 0, 255), 2)
                cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 4)
            else:
                if alarm:
                    alarm.stop()
            status = "Closed" if eyes_closed else "Open"
            cv2.putText(frame, f"Status: {status}", (bbox[0], bbox[1] - 20), font, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Score: {score}", (bbox[0], bbox[1] - 50), font, 0.7, (255, 255, 255), 2)
        except Exception as e:
            print("Prediction error:", e)
            continue

        # Draw bounding box and landmarks
        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (0, 255, 0), 2)
        cv2.circle(frame, (left_eye_x, left_eye_y), 5, (255, 0, 0), -1)
        cv2.circle(frame, (right_eye_x, right_eye_y), 5, (0, 0, 255), -1)
    cv2.imshow("Frame", frame)
    cv2.imshow("Left Eye", left_eye_region)
    cv2.imshow("Right Eye", right_eye_region)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

In [ ]:
cap.release()
cv2.destroyAllWindows()
if alarm:
    mixer.quit()

<br>
from mtcnn import MTCNN<br>
import cv2<br>
import numpy as np<br>
import tensorflow as tf<br>
from pygame import mixer<br>
from collections import deque<br>
<br>
# Load MTCNN detector<br>
detector = MTCNN()<br>
<br>
# Load pre-trained eye closure classification model<br>
# model = tf.keras.models.load_model('finalModel_Augmented.keras')<br>
interpreter = tf.lite.Interpreter(model_path="finalModel_dyn2.tflite")<br>
interpreter.allocate_tensors()<br>
<br>
input_details = interpreter.get_input_details()<br>
output_details = interpreter.get_output_details()<br>
<br>
def predict_eye_state(eye_input):<br>
    interpreter.set_tensor(input_details[0]['index'], eye_input.astype(np.float32))<br>
    interpreter.invoke()<br>
    output_data = interpreter.get_tensor(output_details[0]['index'])  # shape: (1, 2)<br>
    open_prob = output_data[0][0]<br>
    closed_prob = output_data[0][1]<br>
    return open_prob, closed_prob<br>
<br>
<br>
<br>
# Alarm setup<br>
mixer.init()<br>
try:<br>
    alarm = mixer.Sound('alarm.wav')<br>
except:<br>
    print("Warning: alarm.wav not found.")<br>
    alarm = None<br>
<br>
# Parameters<br>
score = 0<br>
SCORE_THRESHOLD = 10<br>
eye_status_history = deque(maxlen=5)<br>
font = cv2.FONT_HERSHEY_SIMPLEX<br>
<br>
def preprocess_eye(img):<br>
    img = cv2.resize(img, (64, 64))<br>
    img = img / 255.0<br>
    img = np.expand_dims(img, axis=0)<br>
    return img<br>
<br>
cap = cv2.VideoCapture(0)<br>
<br>
while True:<br>
    ret, frame = cap.read()<br>
    if not ret:<br>
        break<br>
<br>
    faces = detector.detect_faces(frame)<br>
<br>
    for face in faces:<br>
        bbox = face['box']<br>
        keypoints = face['keypoints']<br>
<br>
        left_eye_x, left_eye_y = keypoints['left_eye']<br>
        right_eye_x, right_eye_y = keypoints['right_eye']<br>
        eye_width = abs(left_eye_x - right_eye_x)<br>
<br>
        # Get ROIs for left and right eye<br>
        lx1 = max(0, int(left_eye_x - eye_width / 2))<br>
        ly1 = max(0, int(left_eye_y - eye_width / 2))<br>
        lx2 = min(frame.shape[1], int(left_eye_x + eye_width / 2))<br>
        ly2 = min(frame.shape[0], int(left_eye_y + eye_width / 2))<br>
<br>
        rx1 = max(0, int(right_eye_x - eye_width / 2))<br>
        ry1 = max(0, int(right_eye_y - eye_width / 2))<br>
        rx2 = min(frame.shape[1], int(right_eye_x + eye_width / 2))<br>
        ry2 = min(frame.shape[0], int(right_eye_y + eye_width / 2))<br>
<br>
        try:<br>
            left_eye_region = frame[ly1:ly2, lx1:lx2]<br>
            right_eye_region = frame[ry1:ry2, rx1:rx2]<br>
<br>
            # left_eye_gray = cv2.cvtColor(left_eye_region, cv2.COLOR_BGR2GRAY)<br>
            # right_eye_gray = cv2.cvtColor(right_eye_region, cv2.COLOR_BGR2GRAY)<br>
<br>
            # Preprocess<br>
            left_input = preprocess_eye(left_eye_region)<br>
            right_input = preprocess_eye(right_eye_region)<br>
<br>
            # lpred = model.predict(left_input, verbose=0)[0][0]<br>
            # rpred = model.predict(right_input, verbose=0)[0][0]<br>
            #<br>
            # l_state = 1 if lpred > 0.5 else 0<br>
            # r_state = 1 if rpred > 0.5 else 0<br>
            l_open, l_closed = predict_eye_state(left_input)<br>
            r_open, r_closed = predict_eye_state(right_input)<br>
<br>
            l_state = 0 if l_closed <= 0.5 else 1  # 1 = open, 0 = closed<br>
            r_state = 0 if r_closed <= 0.5 else 1<br>
<br>
            eyes_closed = (l_state == 0 and r_state == 0)<br>
            eye_status_history.append(1 if eyes_closed else 0)<br>
<br>
            if sum(eye_status_history) >= 4:<br>
                score += 1<br>
            else:<br>
                score -= 1<br>
<br>
            score = max(0, score)<br>
<br>
            # Alarm logic<br>
            if score > SCORE_THRESHOLD:<br>
                if alarm and not mixer.get_busy():<br>
                    alarm.play()<br>
                cv2.putText(frame, "DROWSY!", (10, 60), font, 1, (0, 0, 255), 2)<br>
                cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 4)<br>
            else:<br>
                if alarm:<br>
                    alarm.stop()<br>
<br>
            status = "Closed" if eyes_closed else "Open"<br>
            cv2.putText(frame, f"Status: {status}", (bbox[0], bbox[1] - 20), font, 0.7, (0, 255, 0), 2)<br>
            cv2.putText(frame, f"Score: {score}", (bbox[0], bbox[1] - 50), font, 0.7, (255, 255, 255), 2)<br>
<br>
        except Exception as e:<br>
            print("Prediction error:", e)<br>
            continue<br>
<br>
        # Draw bounding box and landmarks<br>
        cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[0] + bbox[2], bbox[1] + bbox[3]), (0, 255, 0), 2)<br>
        cv2.circle(frame, (left_eye_x, left_eye_y), 5, (255, 0, 0), -1)<br>
        cv2.circle(frame, (right_eye_x, right_eye_y), 5, (0, 0, 255), -1)<br>
<br>
        cv2.imshow("Frame", frame)<br>
        cv2.imshow("Left Eye", left_eye_region)<br>
        cv2.imshow("Right Eye", right_eye_region)<br>
<br>
    if cv2.waitKey(1) & 0xFF == ord("q"):<br>
        break<br>
<br>
cap.release()<br>
cv2.destroyAllWindows()<br>
if alarm:<br>
    mixer.quit()